[reference of preprocess](https://www.pluralsight.com/guides/image-classification-with-pytorch)

In [ ]:
!pip install --no-deps ../input/pretrined-models/timm-0.3.3-py3-none-any.whl

In [ ]:
import os
import pandas as pd
import timm
from PIL import Image, ImageDraw, ImageChops
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from tqdm import tqdm
import numpy as np

In [ ]:
path = "../input/cassava-leaf-disease-classification"
os.listdir(path)

In [ ]:
df = pd.read_csv(path + "/train.csv")

In [ ]:
import json

path_json = '../input/cassava-leaf-disease-classification/label_num_to_disease_map.json'

with open(path_json, mode = 'r') as f:
    label_to_name = json.load(f)

In [ ]:
label_to_name

In [ ]:
df.head()

In [ ]:
# dataframeの写真のIDをpathに変更する
df["path"] = df["image_id"].map(lambda x: path + "/train_images/" + x)
df = df.drop(columns=["image_id"])
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
from sklearn import model_selection

train_df, valid_df = model_selection.train_test_split(
    df, test_size=0.2, random_state=42, stratify=df.label.values
)

In [ ]:
def get_label_id(name):
    id = 0
    if name == 'cbb':
        id = 0
    elif name == 'cbsd':
        id = 1
    elif name == 'cgm':
        id = 2
    elif name == 'healthy':
        id = 4
    return id

In [ ]:
paths, labels = [], []
img_labels = os.listdir('../input/cassava-2019-compe-data/kaggle_upload/train/')
print(img_labels)
for label in img_labels:
    if label != "cmd":
        img_ids = os.listdir("../input/cassava-2019-compe-data/kaggle_upload/train/"+label+"/")
        for img_id in img_ids:
            paths.append("../input/cassava-2019-compe-data/kaggle_upload/train/"+label+"/"+img_id)
            labels.append(get_label_id(label))

In [ ]:
extra_df = pd.DataFrame({'label':labels, 'path':paths})

In [ ]:
extra_df

In [ ]:
train_df = pd.concat([train_df, extra_df])

In [ ]:
train_df.label.value_counts().plot(kind="bar")

In [ ]:
valid_df.label.value_counts().plot(kind="bar")

In [ ]:
train_df = train_df.reset_index().drop(columns=["index"])
train_df.head()

In [ ]:
valid_df = valid_df.reset_index().drop(columns=["index"])
valid_df.head()

In [ ]:
p_df = pd.read_csv('../input/pretrined-models/extra_label.csv', index_col = 0).reset_index().drop(columns = ['index'])

In [ ]:
p_df

In [ ]:
def convert(p):
    tmp = p.replace('[',"").replace(']','').split(',')
    tmp = [float(i) for i in tmp]
    return tmp

In [ ]:
p_df['p_label'] = p_df['p_label'].map(convert)

In [ ]:
p_df

In [ ]:
def add_p(df):
    paths = df['path']
    list_df = pd.DataFrame(columns = ['label', 'path', 'p_label'])
    for path in tqdm(paths):
        tmp = p_df[p_df['path'] == path]
        # print(tmp['p_label'])
        list_df = list_df.append(tmp, ignore_index = True)
    return list_df

In [ ]:
#train_df = add_p(train_df)
#valid_df = add_p(valid_df)

In [ ]:
train_df

In [ ]:
im = Image.open(train_df["path"][0])

In [ ]:
im

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset

from sklearn.model_selection import KFold

import matplotlib.image as img

# データ用の関数の定義

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, dataframe, transform=None, p_label = False):
        super().__init__()
        self.df = dataframe
        self.transform = transform
        self.p = p_label

    def __len__(self):
        return len(self.df["path"])

    def __getitem__(self, index):
        # pathと正解ラベルの入手
        path = self.df["path"][index]
        if self.p:
            label = self.df['p_label'][index]
            label = np.array(label)
        else:
            label = self.df["label"][index]
        # 画像の読み込み
        with open(path, "rb") as f:
            image = Image.open(f)
            image = image.convert("RGB")
        # transformがあるときには、画像に適用する
        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [ ]:
import random

In [ ]:
class make_mask_image:
    def __init__(self, p, mask_size=50):
        self.p = p
        self.mask_size = mask_size

    def __call__(self, image):
        start_width, start_height = [], []
        if random.random() < self.p:
            draw = ImageDraw.Draw(image)
            width, height = image.size
            for i in range(10):
                start_width.append(random.randrange(0, width - self.mask_size))
                start_height.append(random.randrange(0, height - self.mask_size))
            for x, y in zip(start_width, start_height):
                draw.rectangle(
                    (x, y, x + self.mask_size, y + self.mask_size),
                    fill=(0, 0, 0),
                    outline=(0, 0, 0),
                )
        return image

In [ ]:
# 学習データと評価データに対する画像の前処理の定義
image_size = 384
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
train_transform = transforms.Compose(
    [  # 大きさの変更
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomResizedCrop(image_size),
        make_mask_image(p=0.5, mask_size=40),
        # tensor型に変更
        transforms.ToTensor(),
        # 正規化する
        transforms.Normalize(mean=mean, std=std),
    ]
)

valid_transform = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ]
)

In [ ]:
dataset = CassavaDataset(train_df, train_transform, False)

In [ ]:
dataset.__getitem__(3)[1]

# show labels

In [ ]:
class Unnormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std
        
    def __call__(self, tensor):
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
        return tensor

In [ ]:
unnorm = Unnormalize(mean, std)

In [ ]:
def display_img(img, unnorm = None, label = None):
    if unnorm != None:
        img = unnorm(img)
        
    plt.imshow(img.permute(1, 2, 0))
    
    if label != None:
        plt.title(label_to_name[str(label)])

In [ ]:
def display_batch(batch, unnorm = None):
    imgs, labels = batch
    
    if unnorm:
        unnorm_imgs = []
        for img in imgs:
            unnorm_imgs.append(unnorm(img))
        imgs = unnorm_imgs
        
    ig, ax = plt.subplots(figsize=(16, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(imgs, nrow=8).permute(1, 2, 0))

In [ ]:
class TaylorSoftmax(nn.Module):

    def __init__(self, dim=1, n=2):
        super(TaylorSoftmax, self).__init__()
        assert n % 2 == 0
        self.dim = dim
        self.n = n

    def forward(self, x):
        
        fn = torch.ones_like(x)
        denor = 1.
        for i in range(1, self.n+1):
            denor *= i
            fn = fn + x.pow(i) / denor
        out = fn / fn.sum(dim=self.dim, keepdims=True)
        return out

class LabelSmoothingLoss(nn.Module):

    def __init__(self, classes, smoothing=0.0, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.cls = classes 
        self.dim = dim 
    def forward(self, pred, target): 
        """Taylor Softmax and log are already applied on the logits"""
        #pred = pred.log_softmax(dim=self.dim) 
        with torch.no_grad(): 
            true_dist = torch.zeros_like(pred) 
            true_dist.fill_(self.smoothing / (self.cls - 1)) 
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) 
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))
    

class TaylorCrossEntropyLoss(nn.Module):

    def __init__(self, n=2, ignore_index=-1, reduction='mean', smoothing=0.2):
        super(TaylorCrossEntropyLoss, self).__init__()
        assert n % 2 == 0
        self.taylor_softmax = TaylorSoftmax(dim=1, n=n)
        self.reduction = reduction
        self.ignore_index = ignore_index
        self.lab_smooth = LabelSmoothingLoss(num_classes, smoothing=smoothing)

    def forward(self, logits, labels):

        log_probs = self.taylor_softmax(logits).log()
        #loss = F.nll_loss(log_probs, labels, reduction=self.reduction,
        #        ignore_index=self.ignore_index)
        loss = self.lab_smooth(log_probs, labels)
        return loss

In [ ]:
tensor, label = dataset[3]
display_img(tensor, unnorm, label)

In [ ]:
loader = DataLoader(dataset, 16, shuffle = True)
display_batch(next(iter(loader)))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
epoch = 5
batch_size = 16
num_classes = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
resNet = timm.create_model("resnet50", pretrained=False)
resNet.fc = nn.Linear(resNet.fc.in_features, num_classes)
resNet = resNet.to(device)

In [ ]:
ef_model = timm.create_model("tf_efficientnet_b4_ns", pretrained=False)
ef_model.classifier = nn.Linear(ef_model.classifier.in_features, num_classes)
ef_model = ef_model.to(device)

In [ ]:
ef_optimizer = torch.optim.AdamW(ef_model.parameters(), lr=1e-4, weight_decay=0.0001)
ef_scheduler = torch.optim.lr_scheduler.StepLR(ef_optimizer, step_size=2, gamma=0.1)

resNet_optimizer = torch.optim.AdamW(resNet.parameters(), lr=1e-4, weight_decay=0.0001)
resNet_scheduler = torch.optim.lr_scheduler.StepLR(resNet_optimizer, step_size=2, gamma=0.1)
criterion=TaylorCrossEntropyLoss()
# criterion = nn.MSELoss()

In [ ]:
ef_model.load_state_dict(torch.load("../input/models/ef_model.pth", map_location = device))
resNet.load_state_dict(torch.load("../input/models/res_model.pth", map_location = device))

In [ ]:
def collate_fn(batch):
    images, targets = list(zip(*batch))
    images = torch.stack(images)
    targets = torch.Tensor(targets)
    return images, targets

In [ ]:
def calc_correction(model, df):
    model.eval()
    path = df["path"]
    label = df["label"]
    count = 0
    pred_list = [0, 0, 0, 0, 0]
    model = model.to(device)
    with torch.no_grad():
        for i in tqdm(range(len(path))):
            image_path = path[i]
            image_label = label[i]
            image = Image.open(image_path)
            image = valid_transform(image)
            image = image.unsqueeze(0).to(device)
            pred = model(image)
            pred = pred.argmax(1).item()
            pred_list[pred] += 1
            if pred == image_label:
                count += 1
    percent = count / len(path)
    return percent, pred_list

In [ ]:
# lossの表示をする
from matplotlib import pyplot as plt
def plot_losses(epoch, title, train_losses, valid_losses):
    y = list(range(len(train_losses)))
    train_loss = plt.plot(y, train_losses)
    valid_loss = plt.plot(y, valid_losses)
    plt.title(title)
    plt.ylabel("loss")
    plt.legend(
        (train_loss[0], valid_loss[0]), ("train loss", "valid loss"),
    )
    plt.show()


In [ ]:
import time
def train_model(model, dataset, batch_size, optimizer, criterion, scheduler,  epoch, model_title):
    best_model = None
    best_loss = float("inf")
    train_losses, valid_losses = [], []
    kf = KFold(n_splits = 5)
    
    for fold, (train_index, valid_index) in enumerate(kf.split(dataset)):
        print("fold: ", fold)
        train_dataset = Subset(dataset, train_index)
        train_loader = DataLoader(train_dataset, batch_size, shuffle = True, num_workers = 4)#, collate_fn = collate_fn)
        valid_dataset = Subset(dataset, valid_index)
        valid_loader= DataLoader(valid_dataset, batch_size, shuffle = False)#, collate_fn = collate_fn)
        
        for epoch in range(1, epoch + 1):
            epoch_start_time = time.time()
            acc = []
            train_loss = 0
            valid_loss = 0

            model.train()
            for data, target in train_loader:
                data = data.to(device)
                target = target.to(device)
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()*len(data)
            train_loss = train_loss/len(train_loader.sampler)
            train_losses.append(train_loss)

            model.eval()
            for data, target in valid_loader:
                data = data.to(device)
                target = target.to(device)

                with torch.no_grad():
                    output = model(data)
                    # pred = (output.argmax(1) == target)
                    # acc.append(sum(pred)/len(pred))

                    loss = criterion(output, target)

                    valid_loss += loss.item()*len(data)
                    
            if valid_loss < best_loss:
                best_loss = valid_loss
                best_model = model
                    
            scheduler.step()

            # collection = sum(acc)/len(acc)
            valid_loss = valid_loss/len(valid_loader.sampler)
            valid_losses.append(valid_loss)
            print('Time: {:.3f}\t Epoch: {} \tTraining Loss: {:.3f} \tValidation Loss: {:.3f}'# \t Acc: {:.2f}'
                  .format(time.time() - epoch_start_time, epoch, train_loss, valid_loss))#, collection))
            num_collection = []
    torch.save(model.state_dict(), model_title)
    
    return model, train_losses, valid_losses

In [ ]:
def train_models(resNet, ef_model):
    model_title = "./ef_model.pth"
    ef_model, train_losses, valid_losses = train_model(ef_model, dataset, batch_size, ef_optimizer, criterion, ef_scheduler, epoch, model_title)
    print(calc_correction(ef_model, valid_df))
    title = "ef losses"
    plot_losses(epoch, title, train_losses, valid_losses)
    
    model_title = "./res_model.pth"
    resNet, train_losses, valid_losses = train_model(resNet, dataset, batch_size, resNet_optimizer, criterion,resNet_scheduler, epoch, model_title)
    print(calc_correction(resNet, valid_df))
    title = "resNet losses"
    plot_losses(epoch, title, train_losses, valid_losses)

In [ ]:
# train_models(resNet, ef_model)

In [ ]:
class CassaveClassifier(nn.Module):
    def __init__(self, model, ef_model):
        super().__init__()
        self.model = model
        self.ef_model = ef_model
    
    def forward(self, x):
        x1 = self.model(x)
        x2 = self.ef_model(x)
        return (0.5 * x1 + 0.5 * x2)

    def test(self, x, rate):
        x1 = self.model(x)
        x2 = self.ef_model(x)
        p = rate * x1 + (1 - rate) * x2
        return p

In [ ]:
classifier = CassaveClassifier(resNet, ef_model)
classifier = classifier.to(device)

In [ ]:
def test_rate():
    for rate in range(1, 10):
        classifier.eval()
        path = valid_df["path"]
        label = valid_df["label"]
        count = 0
        pred_list = [0, 0, 0, 0, 0]
        for i in tqdm(range(len(path))):
            image_path = path[i]
            image_label = label[i]
            image = Image.open(image_path)
            image = valid_transform(image)
            image = image.unsqueeze(0).to(device)
            pred = classifier.test(image, rate/10).argmax(1).item()
            pred_list[pred] += 1
            if pred == image_label:
                count += 1
        percent = count / len(path)
        print("rate: ", rate/10)
        print("percent: ", percent)

In [ ]:
# test_rate()

In [ ]:
calc_correction(classifier, valid_df)

In [ ]:
path = "../input/cassava-leaf-disease-classification/test_images/"

In [ ]:
image_path = []
image_id = []
for i in os.listdir(path):
    image_id.append(str(i))
    image_path.append(path + str(i))

In [ ]:
pred = []
for path in image_path:
    image = Image.open(path)
    image = valid_transform(image)
    image = image.unsqueeze(0).to(device)
    predict = classifier(image).argmax(1).item()
    pred.append(predict)

In [ ]:
pred

In [ ]:
sub = pd.DataFrame({"image_id": image_id, "label": pred})

In [ ]:
sub

In [ ]:
sub.to_csv("submission.csv", index=False)